# PyTorch 60 minute blitz

## Tensors
See [Pytorch learning the basics](PyTorch Learn the Basics.ipynb#Tensors)

## AutoGrad 
See [Pytorch learning the basics](PyTorch Learn the Basics.ipynb### Automatic differentiation with `torch.autograd`)


## Neural Networks
Examin the example network below for classifying digit images:
![convnet](https://pytorch.org/tutorials/_images/mnist.png "ConvNet")

The typical training procedure is as follows:
1. Define the NN
2. Iterate over the dataset
3. Process the input through the NN
4. Compute the loss
5. Propagate the gradients
6. Update the weights

## Define the network:

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # affine operation: y=wx+b
        self.fc1 = nn.Linear(16*5*5, 120)  # 5x5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify the max_pool with one number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # Flatten all dimension except the batch dimension
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x  # returning logits

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
# The learnable parameters of the model.
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 5, 5])


`torch.nn` only supports mini-bathces, not snigle samples.

For eacmple `nn.Conv2d` takes a 4D Tensor of `nSamples x nChannels x Height x Width`.

If you only have one sample you can use `input.unsqueeze(0)` to add a fake batch dimension

In [13]:
# Random 32x32 input. Note: the expected input size of this network (LeNet) is 32x32. 
# To use this on MNIST you must resize the images.
input = torch.randn(1,32,32).unsqueeze(0)
out = net(input)
print(out)

tensor([[-0.0276,  0.0364, -0.0786, -0.0331, -0.0235, -0.0727, -0.1342,  0.1029,
          0.0127, -0.0410]], grad_fn=<AddmmBackward0>)


In [9]:
# zero out the gradient buffers with random gradients:
net.zero_grad()
out.backward(torch.randn(1,10))

## Computing the loss function, backprop and weight updates

In [15]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.3496, grad_fn=<MseLossBackward0>)


In [16]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0070, -0.0018,  0.0044,  0.0010,  0.0008, -0.0017])


In [17]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update